# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [6]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


In [11]:
df.columns # comando simples para verificar as variáveis

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [9]:
# excluindo as variáveis irrelevantes para o modelo
df =  df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis = 1)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


# 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

In [12]:
#verificando NAs
df.isna().sum()
# há 2573 exclusivamente na variável tempo_emprego o que corresponde a 17%. é muita coisa
# mesmo assim, optei por remove-los para homogeneizar a base e reduzir o tamanho

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [13]:
2573/15000 # percentual de NA na variável tempo_emprego

0.17153333333333334

In [14]:
# removendo NAs
df = df.dropna()
df.isna().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

In [65]:
# criando a lista com todas as variáveis separadas por +. Copiar a saída e colar no modelo
x = df.columns.to_list()
x.pop(-1) # removendo a variável renda 
x = ' + '.join(x)
x

'sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

In [63]:
# criação da matriz design x e Y da df

y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment(3)) + C(educacao, Treatment(1)) + C(estado_civil, Treatment(3)) + C(tipo_residencia, Treatment(2)) + idade + tempo_emprego + qt_pessoas_residencia', df)

modelo = sm.OLS(y,X).fit()

modelo.summary()
# Alteradas as seguintes caselas de referência
# variável tipo_renda com categoria pensionista como mais frequente
# variável educação com categoria Pós graduação como mais frequente
# variável estado_civil com categoria União como mais frequente
# variável tipo_residencia com categoria Com os pais como mais frequente

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Thu, 27 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:38:00   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.3230      0.356     17.749      0.000       5.625       7.021
sexo[T.M]                                             0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                              0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                               0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment(3))[T.Assalariado]            0.3087      0.241      1.280      0.201      -0.164       0.782
C(tipo_renda, Treatment(3))[T.Bolsista]               0.5296      0.341      1.553      0.120      -0.139       1.198
C(tipo_renda, Treatment(3))[T.Empresário]             0.4639      0.241      1.922      0.055      -0.009       0.937
C(tipo_renda, Treatment(3))[T.Servidor público]       0.3663      0.242      1.514      0.130      -0.108       0.841
C(educacao, Treatment(1))[T.Primário]                -0.1071      0.159     -0.673      0.501      -0.419       0.205
C(educacao, Treatment(1))[T.Secundário]              -0.1212      0.142     -0.853      0.394      -0.400       0.157
C(educacao, Treatment(1))[T.Superior completo]       -0.0133      0.142     -0.093      0.926      -0.292       0.265
C(educacao, Treatment(1))[T.Superior incompleto]     -0.1502      0.145     -1.035      0.301      -0.435       0.134
C(estado_civil, Treatment(3))[T.Casado]               0.0343      0.025      1.361      0.174      -0.015       0.084
C(estado_civil, Treatment(3))[T.Separado]             0.3607      0.113      3.182      0.001       0.138       0.583
C(estado_civil, Treatment(3))[T.Solteiro]             0.3015      0.111      2.714      0.007       0.084       0.519
C(estado_civil, Treatment(3))[T.Viúvo]                0.4032      0.118      3.425      0.001       0.172       0.634
C(tipo_residencia, Treatment(2))[T.Aluguel]           0.0258      0.060      0.427      0.670      -0.093       0.144
C(tipo_residencia, Treatment(2))[T.Casa]             -0.0184      0.029     -0.625      0.532      -0.076       0.039
C(tipo_residencia, Treatment(2))[T.Comunitário]      -0.0938      0.095     -0.988      0.323      -0.280       0.092
C(tipo_residencia, Treatment(2))[T.Estúdio]           0.0923      0.088      1.044      0.296      -0.081       0.265
C(tipo_residencia, Treatment(2))[T.Governamental]    -0.0313      0.048     -0.658      0.511      -0.125       0.062
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

- Até então, a transformação das caselas de referência trouxe alguma relevância para algumas variáveis dentro do modelo.
- No entanto, por mais intrigante que possa parecer, ``educação`` não tem relevância no modelo para explicar ``renda``.
- Menos intrigante, ``tipo_residência`` não apresenta um bom potencial explicativo.

# 2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

In [67]:
# remoção da variável menos significante: tipo_residencia

y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment(3)) + C(educacao, Treatment(1)) + C(estado_civil, Treatment(3)) + idade + tempo_emprego + qt_pessoas_residencia', df)

modelo = sm.OLS(y,X).fit()

modelo.summary()

# Alteradas as seguintes caselas de referência
# variável tipo_renda com categoria pensionista como mais frequente
# variável estado_civil com categoria União como mais frequente
# variável educação com categoria Pós graduação como mais frequente

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Thu, 27 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:48:29   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.3070      0.355     17.757      0.000       5.611       7.003
sexo[T.M]                                            0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                             0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                              0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda, Treatment(3))[T.Assalariado]           0.3116      0.241      1.292      0.196      -0.161       0.784
C(tipo_renda, Treatment(3))[T.Bolsista]              0.5335      0.341      1.565      0.118      -0.135       1.202
C(tipo_renda, Treatment(3))[T.Empresário]            0.4672      0.241      1.936      0.053      -0.006       0.940
C(tipo_renda, Treatment(3))[T.Servidor público]      0.3705      0.242      1.532      0.126      -0.104       0.845
C(educacao, Treatment(1))[T.Primário]               -0.1104      0.159     -0.695      0.487      -0.422       0.201
C(educacao, Treatment(1))[T.Secundário]             -0.1200      0.142     -0.845      0.398      -0.399       0.159
C(educacao, Treatment(1))[T.Superior completo]      -0.0124      0.142     -0.087      0.931      -0.291       0.266
C(educacao, Treatment(1))[T.Superior incompleto]    -0.1488      0.145     -1.025      0.305      -0.433       0.136
C(estado_civil, Treatment(3))[T.Casado]              0.0329      0.025      1.310      0.190      -0.016       0.082
C(estado_civil, Treatment(3))[T.Separado]            0.3601      0.113      3.177      0.001       0.138       0.582
C(estado_civil, Treatment(3))[T.Solteiro]            0.3012      0.111      2.713      0.007       0.084       0.519
C(estado_civil, Treatment(3))[T.Viúvo]               0.4008      0.118      3.405      0.001       0.170       0.631
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     3.75e+

- Após a remoção de ``tipo_residencia`` o R-quadrado e R-quadrado ajustado permaneceram o mesmo.
- O AIC caiu de 2719 p 2718e+04
- Houve uma pequena alteração na significância de algumas variáveis
- ``educacao`` ainda se mostra como uma variável não satisfatória

# 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 

In [77]:
# remoção das variáveis sempre que o p=value for menor que 5%

y, X = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + idade + tempo_emprego', df)

modelo = sm.OLS(y,X).fit()

modelo.summary()

# Alteradas as seguintes caselas de referência
# variável tipo_renda com categoria pensionista como mais frequente
# variável estado_civil com categoria União como mais frequente

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Thu, 27 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:57:31   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    7.2550      0.032    228.981      0.000       7.193       7.317
sexo[T.M]                    0.7700      0.015     52.851      0.000       0.741       0.799
posse_de_veiculo[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
posse_de_imovel[T.True]      0.0880      0.014      6.379      0.000       0.061       0.115
idade                        0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         209.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Após remover diversas variáveis qualitativas ``tipo_renda + educacao + estado_civil + tipo_residencia + qtd_pessoas_residencia + qtd_filhos`` o R-quadrado diminuiu para 0.346, como esperado.
- No entato o AIC subiu de 2.718e+04 para 2.736e+04.
- Importante mencionar que a última variável a ser removida foi a ``qtd_filhos``, uma vez que ela teve um p-value de 0.06,
- Sendo estes os parâmetros até então usados para avaliar o modelo: p-value, r-quadrado e AIC; o modelo apresenta todas as variáveis que são relevantes para a modelagem, as quais são apenas 5 de 16 originalmente.
- De certa forma, as covariantes apresentam-se relevantes, todas com p-value igual a 0. No entanto todos os parâmetros sofreram modificações negativas com menor potencial explicativo.